In [1]:
from finlab import login, data

# 登入 FinLab (請填入您在 FinLab 網站申請的 API token)
login('TsQjZ57Cu9ZM7IrQHw30NsuzJ6qgWh6VT7oX717CEJKTlAugseXgJLwVsFMXZTCE#free')

輸入成功!


In [3]:
from finlab import data

# 取得所有股票的「收盤價」資料（資料格式為 pandas DataFrame）
open_prices = data.get('price:開盤價')
high_prices = data.get('price:最高價')
low_prices = data.get('price:最低價')
close_prices = data.get('price:收盤價')
volume = data.get('price:成交股數')

Daily usage: 238.3 / 500 MB - price:成交股數


In [4]:
import pandas as pd

# open, high, low, close, volume 都是 DataFrame
# 索引(index)=日期, 欄位(columns)=股票代號

df_open = open_prices.stack().rename('open')
df_high = high_prices.stack().rename('high')
df_low  = low_prices.stack().rename('low')
df_close= close_prices.stack().rename('close')
df_vol  = volume.stack().rename('volume')


df_ohlcv_long = pd.concat([df_open, df_high, df_low, df_close, df_vol], axis=1)

result_dict = {}
for sym, subdf in df_ohlcv_long.groupby(level='symbol'):
    # subdf 的索引是 (date, symbol)，先把 symbol 這層移除
    subdf = subdf.droplevel('symbol')
    # 可以順便按照日期排序
    subdf = subdf.sort_index()
    # 存到字典 result_dict 裡，以股票代號 sym 當 key
    result_dict[sym] = subdf

# DataFrame (index=日期, columns=[open, high, low, close, volume])

In [10]:
import pickle

with open(r'ALL_stock_DATE_OHLC_dict.pkl', 'wb') as f:
    pickle.dump(result_dict, f)

In [12]:
for sym, df_ohlc in result_dict.items():
    csv_filename = fr"ALL_stock_DATE_OHLC_dict/{sym}.csv"
    df_ohlc.to_csv(csv_filename)